# Ambiente

### Instalar librerías y reiniciar kernel.

In [1]:
pip install mysql-connector

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas

     |████████████████████████████████| 9.7 MB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 15.4 MB 2.4 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pyarrow

     |████████████████████████████████| 20.7 MB 6.7 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


### Importaciones

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy import *
#import json
#import pyarrow.parquet as pq
#import pyarrow

### Crear acceso a engine de mysql e inspeccionar.

In [2]:
engine = create_engine('mysql+mysqlconnector://test:test123@db/test', echo = True)

In [3]:
pd.read_sql("select current_date", con=engine)

2021-02-04 06:22:54,079 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2021-02-04 06:22:54,084 INFO sqlalchemy.engine.base.Engine {}
2021-02-04 06:22:54,096 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2021-02-04 06:22:54,101 INFO sqlalchemy.engine.base.Engine {}
2021-02-04 06:22:54,114 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2021-02-04 06:22:54,117 INFO sqlalchemy.engine.base.Engine {}
2021-02-04 06:22:54,129 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2021-02-04 06:22:54,134 INFO sqlalchemy.engine.base.Engine {}
2021-02-04 06:22:54,146 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2021-02-04 06:22:54,150 INFO sqlalchemy.engine.base.Engine {}
2021-02-04 06:22:54,162 INFO sqlalchemy.engine.base.Engine DESCRIBE `select current_date`
2021-02-04 06:22:54,167 INFO sqlalchemy.engine.base.Engine {}
2021-02-04 06:22:54,183 INFO sqlalc

,current_date
0,2021-02-04


In [4]:
inspector = inspect(engine)
schemas = inspector.get_schema_names()

for schema in schemas:
    print("schema: %s" % schema)
    for table_name in inspector.get_table_names(schema=schema):
        for column in inspector.get_columns(table_name, schema=schema):
            print("Column: %s" % column)

2021-02-04 06:22:57,882 INFO sqlalchemy.engine.base.Engine SHOW schemas
2021-02-04 06:22:57,884 INFO sqlalchemy.engine.base.Engine {}
schema: information_schema
2021-02-04 06:22:57,891 INFO sqlalchemy.engine.base.Engine SHOW FULL TABLES FROM `information_schema`
2021-02-04 06:22:57,893 INFO sqlalchemy.engine.base.Engine {}
schema: test
2021-02-04 06:22:57,904 INFO sqlalchemy.engine.base.Engine SHOW FULL TABLES FROM `test`
2021-02-04 06:22:57,906 INFO sqlalchemy.engine.base.Engine {}


In [5]:
print(engine.table_names())

2021-02-04 06:23:01,017 INFO sqlalchemy.engine.base.Engine SHOW FULL TABLES FROM `test`
2021-02-04 06:23:01,021 INFO sqlalchemy.engine.base.Engine {}
[]


# ETL 4

## Extraer

In [ ]:
#table = pq.read_table('data.parquet')
#table.to_pandas()

In [ ]:
df = pd.read_parquet('data.parquet')

In [ ]:
df.dtypes

## Transformar

In [ ]:
u = df.select_dtypes(object)
df[u.columns] = u.apply(
    lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))

## Load al engine

In [ ]:
create_table_query = pd.io.sql.get_schema(df.reset_index(), 'test.data_parquet', con=engine)
print(create_table_query)
type(create_table_query)

In [ ]:
with open("data_parquet.sql", "w") as text_file:
    text_file.write(create_table_query)

In [ ]:
if not engine.dialect.has_table(engine, "test.data_parquet"):  # If table don't exist, Create.
    print("test.data_parquet no existe")
    df.to_sql('data_parquet', con=engine, schema="test", if_exists='append')

In [ ]:
with engine.begin() as transaction:
    transaction.execute('DELETE FROM test.data_parquet')
    df.to_sql('data_parquet', con=transaction, schema='test', if_exists='append', index=False)